# Train Pretrained Vertical Model for oral bioavailability prediction

1. This notebook used the verticalGNN model pretrained with high similarity solubility dataset
2. Ensure that the notebook, data folder, config.py, engine.py, model.py and utils.py are in the same directory. 
3. Whole process can be repeated by running the cells in order
4. Otherwise, download the saved models as described in the README.md from google drive, comment the run_training line to get the results in the paper.

In [1]:
# import all required materials

import torch
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
import os

from model import VerticalGNN
from config import NUM_FEATURES, NUM_TARGET, EDGE_DIM, DEVICE, SEED_NO, PATIENCE, EPOCHS, NUM_GRAPHS_PER_BATCH, N_SPLITS, best_params_vertical
from engine import EngineSol, EngineHOB
from utils import seed_everything, LoadHOBDataset, LoadSolDataset

First, we define a train and test function that can aid us in transfer learning. Then, we evaluate how different factors can affect the results of transfer learning. 

In [2]:
def run_training(method_tf, train_loader, valid_loader, params,es_trigger, path_to_pretrained_model, path_to_save_trained_model):
    
    '''
    Define a function to wrap training

    Args:
    method_tf (str): freeze --> freeze parameters of feature extraction block, fine_tune_5x -> fine tune at 5x slower learning rate, 
                    fine_tune_10x -> fine tune at 10x slower learning rate
    train_loader: DataLoader class from pytorch geometric containing train data
    valid_loader: DataLoader class from pytorch geometric containing validation data
    params (dict): dictionary containing the hyperparameters
    es_trigger (int): a number to force train model before triggering early stopping mechanism 
    path_to_pretrained_model (str): path to load the pretrained models
    path_to_save_trained_model: path to save the trained models

    Return:
    best loss: return best validation loss
    '''
    
    model = VerticalGNN(
            num_features=NUM_FEATURES,
            num_targets=NUM_TARGET,
            num_gin_layers=params["num_gin_layers"],
            num_graph_trans_layers=params["num_graph_trans_layers"],
            hidden_size=params["hidden_size"],
            n_heads=params["n_heads"],
            dropout=params["dropout"],
            edge_dim=EDGE_DIM,
        )

    model.load_state_dict(torch.load(path_to_pretrained_model))  
    model.to(DEVICE)
    if method_tf == 'freeze':
        for param in model.gin_model.parameters():
            param.requires_grad=False
        for param in model.graph_trans_model.parameters():
            param.requires_grad=False

        optimizer=torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = params['learning_rate'])
    
    elif method_tf == 'fine_tune_5x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])

    elif method_tf == 'fine_tune_10x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])

    elif method_tf == 'fine_tune':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
        
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(
            f"Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}"
        )
        if epoch+1>es_trigger:
            if valid_loss < best_loss:
                best_loss = valid_loss
                early_stopping_counter = 0  # reset counter
                print("Saving model...")
                torch.save(model.state_dict(), path_to_save_trained_model)
            else:
                early_stopping_counter += 1

            if early_stopping_counter > early_stopping_iter:
                print("Early stopping...")
                break
            print(f"Early stop counter: {early_stopping_counter}")

    return best_loss



In [4]:
def run_testing(method_tf, test_loader, params, path_to_trained_model):
    
    '''
    Define a function to wrap testing

    Args:
    method_tf (str): freeze --> freeze parameters of feature extraction block, fine_tune_5x -> fine tune at 5x slower learning rate, 
                    fine_tune_10x -> fine tune at 10x slower learning rate
    test_loader: DataLoader class from pytorch geometric containing test data
    params (dict): dictionary containing the hyperparameters
    path_to_save_trained_model: path to load the saved trained models

    Return:
    bce: logloss from pytorch geometric 
    acc: accuracy score
    f1: f1 score
    roc_auc: roc auc score
    '''
    
    model = VerticalGNN(
            num_features=NUM_FEATURES,
            num_targets=NUM_TARGET,
            num_gin_layers=params["num_gin_layers"],
            num_graph_trans_layers=params["num_graph_trans_layers"],
            hidden_size=params["hidden_size"],
            n_heads=params["n_heads"],
            dropout=params["dropout"],
            edge_dim=EDGE_DIM,
        )

    model.load_state_dict(torch.load(path_to_trained_model))  
    model.to(DEVICE)
    if method_tf == 'freeze':
        for param in model.gin_model.parameters():
            param.requires_grad=False
        for param in model.graph_trans_model.parameters():
            param.requires_grad=False

        optimizer=torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = params['learning_rate'])        
    
    elif method_tf == 'fine_tune_5x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
        

    elif method_tf == 'fine_tune_10x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
        
    elif method_tf == 'fine_tune':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
        

    eng = EngineHOB(model, optimizer, device=DEVICE)
    bce, acc, f1, roc_auc = eng.test(test_loader)
    print(f"bce:{bce}, acc :{acc}, f1: {f1}, roc_auc: {roc_auc}")
    return bce, acc, f1, roc_auc

# Effect of number of pre-training epochs to transfer learning prediction performance
1. Weights are first frozen for the feature extraction block. 
2. Models are allowed to train and only weights for the classifier block is allowed to be updated. 
3. To evaluate how number of pre-training epochs affect the transfer learning prediction performance.

### Note on path
1. path_to_pretrained_model = path to model pretrained with solubility dataset
2. path_to_save_trained_model = path to pretrained model trained with oral bioavailability dataset
3. path_to_trained_model = path to pretrained model trained with oral bioavailability dataset

In [4]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'freeze'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_20/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_20/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_20_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.471498504281044, validation loss : 0.7738953828811646
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7841273695230484, validation loss : 0.6364569664001465
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.8474976122379303, validation loss : 0.6353843212127686
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.708319678902626, validation loss : 0.7124544978141785
Early stop counter: 1
Epoch: 5/300, train loss : 0.7195297926664352, validation loss : 0.7489401698112488
Early stop counter: 2
Epoch: 6/300, train loss : 0.7128502428531647, validation loss : 0.6932723522186279
Early stop counter: 3
Epoch: 7/300, train loss : 0.6873944997787476, validation loss : 0.6446869373321533
Early stop counter: 4
Epoch: 8/300, train loss : 0.6795462518930435, validation loss : 0.6507980227470398
Early stop counter: 5
Epoch: 9/300, train loss : 0.6710593104362488, validation loss : 0.6645365953445435
Ea

In [5]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'freeze'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_40/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_40/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_40_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.5485136806964874, validation loss : 0.8068264126777649
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7894892543554306, validation loss : 0.6331623792648315
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.8399862349033356, validation loss : 0.6155871748924255
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.7049555778503418, validation loss : 0.6887074112892151
Early stop counter: 1
Epoch: 5/300, train loss : 0.7112648040056229, validation loss : 0.7233129143714905
Early stop counter: 2
Epoch: 6/300, train loss : 0.6978248059749603, validation loss : 0.66497802734375
Early stop counter: 3
Epoch: 7/300, train loss : 0.676585853099823, validation loss : 0.6229011416435242
Early stop counter: 4
Epoch: 8/300, train loss : 0.6720149666070938, validation loss : 0.6215425729751587
Early stop counter: 5
Epoch: 9/300, train loss : 0.6653987765312195, validation loss : 0.6354700326919556
Ear

In [6]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'freeze'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )
        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.5550463497638702, validation loss : 0.8062240481376648
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.8117981106042862, validation loss : 0.6435795426368713
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.8472504764795303, validation loss : 0.6071640253067017
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.704051062464714, validation loss : 0.6731581687927246
Early stop counter: 1
Epoch: 5/300, train loss : 0.7091736197471619, validation loss : 0.7052314877510071
Early stop counter: 2
Epoch: 6/300, train loss : 0.692707970738411, validation loss : 0.6425475478172302
Early stop counter: 3
Epoch: 7/300, train loss : 0.6699083149433136, validation loss : 0.6056686639785767
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6647282838821411, validation loss : 0.6068962216377258
Early stop counter: 1
Epoch: 9/300, train loss : 0.652254581451416, validation loss : 0.6212

## Next, we evaluated other methods for trf learning 
1. Using the best performance model --> pretrained at 60 epochs using high similarity data
2. We investigated the prediction performances by using different learning rate at the feature extraction block. 

## Fine tuning at 5x slower learning rate

In [11]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_5x'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.3573160022497177, validation loss : 0.6672883033752441
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.743910402059555, validation loss : 0.6624475717544556
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.7037111967802048, validation loss : 0.7322511076927185
Early stop counter: 1
Epoch: 4/300, train loss : 0.6863258332014084, validation loss : 0.6494010090827942
Saving model...
Early stop counter: 0
Epoch: 5/300, train loss : 0.6629908531904221, validation loss : 0.6637245416641235
Early stop counter: 1
Epoch: 6/300, train loss : 0.6509633809328079, validation loss : 0.6499987244606018
Early stop counter: 2
Epoch: 7/300, train loss : 0.6347473412752151, validation loss : 0.6041989922523499
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6238701641559601, validation loss : 0.6014638543128967
Saving model...
Early stop counter: 0
Epoch: 9/300, train loss : 0.6110958158969879, valida

## Fine tuning at 10x slower learning rate

In [12]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_10x'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.4461365640163422, validation loss : 0.7175863981246948
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7797985076904297, validation loss : 0.6168623566627502
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.7397231608629227, validation loss : 0.6593972444534302
Early stop counter: 1
Epoch: 4/300, train loss : 0.6920460760593414, validation loss : 0.724404513835907
Early stop counter: 2
Epoch: 5/300, train loss : 0.6735002398490906, validation loss : 0.657353937625885
Early stop counter: 3
Epoch: 6/300, train loss : 0.6436703950166702, validation loss : 0.6133093237876892
Saving model...
Early stop counter: 0
Epoch: 7/300, train loss : 0.6438823640346527, validation loss : 0.6146202087402344
Early stop counter: 1
Epoch: 8/300, train loss : 0.6264294385910034, validation loss : 0.6385821104049683
Early stop counter: 2
Epoch: 9/300, train loss : 0.6147092431783676, validation loss : 0.6336943507194519
Ea

### Fine tune with normal learning rate

In [4]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.0868065059185028, validation loss : 0.7383393049240112
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.6847097873687744, validation loss : 0.6526279449462891
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.6485132575035095, validation loss : 0.6252182126045227
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.636753499507904, validation loss : 0.6638116240501404
Early stop counter: 1
Epoch: 5/300, train loss : 0.6230585128068924, validation loss : 0.5980041027069092
Saving model...
Early stop counter: 0
Epoch: 6/300, train loss : 0.6340309530496597, validation loss : 0.5991331934928894
Early stop counter: 1
Epoch: 7/300, train loss : 0.6397128105163574, validation loss : 0.6701903343200684
Early stop counter: 2
Epoch: 8/300, train loss : 0.6281179487705231, validation loss : 0.5973500609397888
Saving model...
Early stop counter: 0
Epoch: 9/300, train loss : 0.6206066906452179, valida

# Evaluating the use of slower learning rate + force training the model 
1. Force training at 10, 15 and 20 epochs

In [5]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_5x'
params = best_params_vertical
es_trigger = 10
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.357315942645073, validation loss : 0.6672882437705994
Epoch: 2/300, train loss : 0.7439092099666595, validation loss : 0.6624501943588257
Epoch: 3/300, train loss : 0.7037086337804794, validation loss : 0.7322496771812439
Epoch: 4/300, train loss : 0.686323881149292, validation loss : 0.6494119167327881
Epoch: 5/300, train loss : 0.6629830002784729, validation loss : 0.6637389659881592
Epoch: 6/300, train loss : 0.6509374380111694, validation loss : 0.6498057842254639
Epoch: 7/300, train loss : 0.6347090005874634, validation loss : 0.6043737530708313
Epoch: 8/300, train loss : 0.6237917244434357, validation loss : 0.6012130975723267
Epoch: 9/300, train loss : 0.6110760569572449, validation loss : 0.6022387146949768
Epoch: 10/300, train loss : 0.6114949584007263, validation loss : 0.598514199256897
Epoch: 11/300, train loss : 0.5964422672986984, validation loss : 0.6066474914550781
Saving model...
Early stop counter: 0
Epoch: 12/300, trai

In [6]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_5x'
params = best_params_vertical
es_trigger = 15
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.3573160022497177, validation loss : 0.6672883033752441
Epoch: 2/300, train loss : 0.7439104169607162, validation loss : 0.662447452545166
Epoch: 3/300, train loss : 0.7037108689546585, validation loss : 0.7322039008140564
Epoch: 4/300, train loss : 0.6863123774528503, validation loss : 0.649365246295929
Epoch: 5/300, train loss : 0.6630030423402786, validation loss : 0.6638603210449219
Epoch: 6/300, train loss : 0.6510104387998581, validation loss : 0.6501813530921936
Epoch: 7/300, train loss : 0.6346969306468964, validation loss : 0.6044291853904724
Epoch: 8/300, train loss : 0.623812273144722, validation loss : 0.601441502571106
Epoch: 9/300, train loss : 0.6109722703695297, validation loss : 0.6025055050849915
Epoch: 10/300, train loss : 0.6113825738430023, validation loss : 0.5986625552177429
Epoch: 11/300, train loss : 0.5966213494539261, validation loss : 0.6058651208877563
Epoch: 12/300, train loss : 0.598927840590477, validation 

In [7]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_5x'
params = best_params_vertical
es_trigger = 20
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/high/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/high/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.3573160022497177, validation loss : 0.6672883629798889
Epoch: 2/300, train loss : 0.7439104169607162, validation loss : 0.6624475717544556
Epoch: 3/300, train loss : 0.7037111818790436, validation loss : 0.7322512269020081
Epoch: 4/300, train loss : 0.686325803399086, validation loss : 0.6494013667106628
Epoch: 5/300, train loss : 0.6629908829927444, validation loss : 0.6637232303619385
Epoch: 6/300, train loss : 0.6509624123573303, validation loss : 0.6499991416931152
Epoch: 7/300, train loss : 0.6347365975379944, validation loss : 0.6042731404304504
Epoch: 8/300, train loss : 0.62388676404953, validation loss : 0.6013971567153931
Epoch: 9/300, train loss : 0.6111169010400772, validation loss : 0.6026694178581238
Epoch: 10/300, train loss : 0.6115411669015884, validation loss : 0.5990117788314819
Epoch: 11/300, train loss : 0.5965953469276428, validation loss : 0.6059995293617249
Epoch: 12/300, train loss : 0.5988588035106659, validatio